In [1]:
# 读取fileList
import numpy as np

fileList = np.load("/home/wngys/lab/DeepFold/protein_infor/entPathArray.npy", allow_pickle=True).tolist()

In [ ]:
# # GitHub 原代码 生成距离矩阵
# import sys
# import os
# import numpy
# import scipy
# import scipy.spatial

# from Bio.PDB import PDBParser


# def get_distance_matrix(pdb_path):
#     parser = PDBParser()
#     structure = parser.get_structure('structure', pdb_path).get_list()[0]
#     residue_positions = get_residue_positions(structure)
#     pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
#     pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
#     return pdb_dist_mat


# def get_residue_ids(structure):
#     ids = [r.get_id()[1] for r in structure.get_residues()]
#     return ids


# def get_residue_positions(structure):
#     residue_ids = get_residue_ids(structure)
#     positions = numpy.ones((residue_ids[-1] - residue_ids[0] + 1, 3)) * float('inf')
#     for residue in structure.get_residues():
#         atoms = residue.get_atoms()
#         for a in atoms:
#             if a.get_name() == 'CA':
#                 positions[residue.get_id()[1] - residue_ids[0]] = a.get_coord()

#     return positions

In [2]:
# 修改的 GitHub 原代码 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

from Bio.PDB import PDBParser


def get_distance_matrix_modified(pdb_path):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path).get_list()[0]
    residue_positions = get_residue_positions(structure)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
    pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
    return pdb_dist_mat


def get_residue_ids(structure):
    ids = [r.get_id()[1] for r in structure.get_residues()]
    print(ids)
    print(len(ids))
    return ids


def get_residue_positions(structure):
    residue_ids = get_residue_ids(structure)
    positions = numpy.ones((len(residue_ids), 3)) * float('inf')
    i = 0
    for residue in structure.get_residues():
        atoms = residue.get_atoms()
        for a in atoms:
            if a.get_name() == 'CA':
                positions[i] = a.get_coord()
                i = i + 1
    return positions

In [3]:
# 测试改进后的获取距离矩阵的函数

pdb_path = "/home/wngys/scop/scope-2.07-40/bd/d1bd7a_.ent"
parser = PDBParser()
structure = parser.get_structure('structure', pdb_path).get_list()[0]

get_residue_ids(structure)
# residue_positions = get_residue_positions(structure)

[87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 70, 71, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
176


/home/wngys/.local/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1428
  warnings.warn(


[87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 -1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 70,
 71,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82]

In [4]:
# # 自己解析 ent文件 生成距离矩阵
# import sys
# import os
# import numpy
# import scipy
# import scipy.spatial

# def get_distance_matrix_mine(pdb_path):
#     CA_positions = []
#     with open(pdb_path, 'r') as f:
#         while True:
#             line = f.readline()
#             if not line:
#                 break
#             if line[:4] != "ATOM":
#                 if line[:6] != "HETATM":
#                     continue
#             if line[13:15] == "CA":
#                 position = [float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())]
#                 CA_positions.append(position)
#     pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
#     return pdb_dist_mat

In [2]:
# 自己解析 ent文件 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

def get_distance_matrix_mine(pdb_path):
    CA_positions = []
    residue_set = set()
    with open(pdb_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line[:4] != "ATOM":
                if line[:6] != "HETATM":
                    continue
            if line[13:15] == "CA":
                residue_id = line[22:27].strip()
                if residue_id not in residue_set:
                    position = [float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())]
                    CA_positions.append(position)
                    residue_set.add(residue_id)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
    return pdb_dist_mat

In [6]:
# 生成14000多个距离矩阵并保存（版本2：解决）
import warnings
warnings.filterwarnings('ignore')
import numpy as np

errorList = []
errorFile = []

# save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_inf/"
save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_mine/"
# save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_modified/"

i = 0

for file_n in fileList:
    i = i + 1
    if i % 100 == 0:
        print(i)
    
    ID = file_n.split("/")[-1].split(".")[0]
    
    try:
        # M = get_distance_matrix(file_n)
        M = get_distance_matrix_mine(file_n)
        # M = get_distance_matrix_modified(file_n)
    except:
        errorList.append(ID)
        errorFile.append(file_n)
        print("error: " + ID)
    else:
        np.save(save_dir+ID+".npy", M)
        # print("done: " + ID)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200


In [8]:
print(len(errorList))
print(len(errorFile))

0
0


In [ ]:
# get_distance_matrix_mine、get_distance_matrix_modified未产生报错的文件
# np.save("/home/wngys/lab/DeepFold/protein_infor/errorIDArrayMine.npy", errorList)
# np.save("/home/wngys/lab/DeepFold/protein_infor/errorEntPathArrayMine.npy", errorFile)

In [2]:
errorList = ['d1nria_', 'd3msza1', 'd1ngvb_', 'd1bd7a_', 'd4ax1b_', 'd1nova_']
errorFile = ['/home/wngys/scop/scope-2.07-40/nr/d1nria_.ent', 
'/home/wngys/scop/scope-2.07-40/ms/d3msza1.ent', 
'/home/wngys/scop/scope-2.07-40/ng/d1ngvb_.ent', 
'/home/wngys/scop/scope-2.07-40/bd/d1bd7a_.ent', 
'/home/wngys/scop/scope-2.07-40/ax/d4ax1b_.ent', 
'/home/wngys/scop/scope-2.07-40/no/d1nova_.ent']

np.save("/home/wngys/lab/DeepFold/protein_infor/errorIDArrayInf.npy", errorList)
np.save("/home/wngys/lab/DeepFold/protein_infor/errorEntPathArrayInf.npy", errorFile)

In [6]:
# 测试：加载举例矩阵
M_2 = np.load("/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_modified/d1tqga1.npy", allow_pickle=True)
print(M_2.shape)
print(M_2)

(102, 102)
[[ 0.          3.80693236  6.81197329 ... 45.78236534 47.1932875
  49.68702315]
 [ 3.80693236  0.          3.80311334 ... 46.52298159 48.18525723
  50.67316593]
 [ 6.81197329  3.80311334  0.         ... 44.27025018 46.15558697
  48.58935649]
 ...
 [45.78236534 46.52298159 44.27025018 ...  0.          3.79448412
   5.33901211]
 [47.1932875  48.18525723 46.15558697 ...  3.79448412  0.
   3.82088434]
 [49.68702315 50.67316593 48.58935649 ...  5.33901211  3.82088434
   0.        ]]
